In [0]:
!pip install -U -q pydrive

In [87]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import files
from oauth2client.client import GoogleCredentials
import pandas as pd
import io
from scipy.sparse import csr_matrix
import numpy as np
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer,PorterStemmer
import nltk
import re
import pprint
from collections import Counter
import string
from nltk.tokenize import RegexpTokenizer, sent_tokenize, word_tokenize
from nltk import pos_tag
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.feature_selection import mutual_info_classif
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.compose import ColumnTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import classification_report, accuracy_score, f1_score
from string import punctuation
from nltk.util import ngrams
from nltk.corpus import stopwords



nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [0]:
stop_words = set(stopwords.words('english'))

In [0]:
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [5]:
uploaded = files.upload()

Saving yelp2.csv to yelp2.csv


In [6]:
yelp = pd.read_csv(io.StringIO(uploaded['yelp2.csv'].decode('utf-8')))
len(yelp)

99579

In [7]:
uploaded = files.upload()

Saving negation_list.csv to negation_list.csv


In [0]:
negate = pd.read_csv(io.StringIO(uploaded['negation_list.csv'].decode('utf-8')))

In [9]:
uploaded = files.upload()

Saving positivewords.csv to positivewords.csv


In [0]:
pos_words = pd.read_csv(io.StringIO(uploaded['positivewords.csv'].decode('utf-8')))

In [11]:
uploaded = files.upload()

Saving negativewords.csv to negativewords.csv


In [0]:
neg_words = pd.read_csv(io.StringIO(uploaded['negativewords.csv'].decode('utf-8')))

In [0]:
yelp = pd.read_csv('yelp2.csv')
negate = pd.read_csv('negation_list.csv')
pos_words = pd.read_csv('positivewords.csv')
neg_words = pd.read_csv('negativewords.csv')

In [90]:
yelp.head()

,Unnamed: 0,review_id,user_id,business_id,stars,useful,funny,cool,text,date,positive,lang
0,4928306,7TWd_t1xe5YQX1f20MdXEg,X20U1YunHVHhqiQe3rkJ-Q,WAIEdzllzVnSwcTE7GEn6A,4.0,1,0,0,I went there because my front bumper fell off ...,2015-07-22 21:14:56,1,en
1,7160,XhX-U8N-sY6v5VaaxU2IUg,xR4engcg6Vm8xToBgKI5FA,pVCF9rIPF4y697QeRMpweA,4.0,1,0,0,"Nice, clean, large grocery store with great va...",2014-02-05 23:10:33,1,en
2,1675101,O6CDUki8r7yYcQOwO4kY9w,xzdEPPOsLzRiHN9Z3nPKag,9xdVIIzHp12Zrox5aFaaHA,4.0,5,0,1,"Ladies and Gentlemen, we are officially witnes...",2011-08-10 01:30:52,1,en
3,5702417,14p07vYeRrEvOt7MrOp-Vg,zWWcik1fRPZviBCQLC26FQ,IZ33QKVggRtHdMdWoDPi6Q,4.0,1,0,0,I think this place deserves a much better rati...,2016-09-23 21:18:34,1,en
4,753841,xnRBYPoavUMA_ept3VZ86A,jUZW1alRK8QvdZ-pH1OquQ,cOJ1uIVIHCiefUyWG2wDfw,4.0,8,3,5,Never heard of this place. Really cool spot! ...,2014-05-28 03:04:02,1,en


In [0]:
yelp = yelp.rename(columns={'positive':'pos_class'})

In [0]:
yelp = yelp.drop(['Unnamed: 0','lang','useful','stars','review_id','user_id','business_id','cool','funny','date'],axis=1)

In [0]:
def lemmatize_text(text):
    lemmatizer = nltk.stem.WordNetLemmatizer()
    words = nltk.word_tokenize(text)
    words=[word.lower() for word in words if word.isalpha()]
    return [lemmatizer.lemmatize(w, pos='v') for w in words]

#function to remove # and numbers from General Inquirer positive and negative word lists
def clean_list(mylist):
    import re
    lis = []
    for word in mylist:
        word = word.lower()
        word = re.sub('[#0-9]','', word)    
        lis.append(word)
    myset = set(lis)
    cleaned_list = list(myset)
    return cleaned_list

In [94]:
#clean \n out of text
clean_text = []
for t in tqdm(yelp['text']):
    temp = []
    t = t.replace('\n','')
    temp.append(t)
    clean_text.append(temp)
clean_text = pd.DataFrame(clean_text, columns=['clean_text'])

100%|██████████| 99579/99579 [00:00<00:00, 628067.06it/s]


In [95]:
yelp = yelp.join(clean_text)
yelp.head()

,text,pos_class,clean_text
0,I went there because my front bumper fell off ...,1,I went there because my front bumper fell off ...
1,"Nice, clean, large grocery store with great va...",1,"Nice, clean, large grocery store with great va..."
2,"Ladies and Gentlemen, we are officially witnes...",1,"Ladies and Gentlemen, we are officially witnes..."
3,I think this place deserves a much better rati...,1,I think this place deserves a much better rati...
4,Never heard of this place. Really cool spot! ...,1,Never heard of this place. Really cool spot! ...


In [0]:
#lists i pulled of negation words, positive words, and negative words from general inquirer
# negate = pd.read_csv('C:/Users/Gabo/Desktop/PoC_Resit/yelp-dataset/negation_list.csv')
negate = negate['Column1'].to_list() 
negate = clean_list(negate)

# pos_words = pd.read_csv('C:/Users/Gabo/Desktop/PoC_Resit/yelp-dataset/positivewords.csv')
pos_words = pos_words['Column1'].to_list() 
pos_words = clean_list(pos_words)

# neg_words = pd.read_csv('C:/Users/Gabo/Desktop/PoC_Resit/yelp-dataset/negativewords.csv')
neg_words = neg_words['Column1'].to_list() 
neg_words = clean_list(neg_words)

In [97]:
#create lemmas feature
lemmas = [lemmatize_text(yelp.text[x]) for x in tqdm(range(len(yelp['text']))) ]

100%|██████████| 99579/99579 [02:47<00:00, 594.57it/s]


In [0]:
yelp['lemmas'] = lemmas

In [99]:
#create binary feature of if negations are present
negations = []
for word in tqdm(yelp['lemmas']):
    count =0
    for w in word:
        if w.lower() in negate:
            count+=1
    if count >= 1:
        negations.append(1)
    else:
        negations.append(0)
            

100%|██████████| 99579/99579 [00:29<00:00, 3402.46it/s]


In [0]:
yelp['negate'] = negations

In [0]:
one_hot_text = []
for t in yelp['lemmas']:
    string = ""
    for i in t:
        temp = []
        string = string + " " + i
    temp.append(string)
    one_hot_text.append(temp)

In [0]:
one_hot_text_yelp = pd.DataFrame(one_hot_text, columns=['text_one_hot'])

In [0]:
yelp = yelp.join(one_hot_text_yelp)

In [104]:

ps = PorterStemmer()
for i,r in tqdm(yelp.iterrows(), total=yelp.shape[0]):
    text = r['text']
    counts = Counter(text)
    punctuation_counts = {k:v for k, v in counts.items() if k in punctuation}
    pc = sum(punctuation_counts.values())
    yelp.at[i, 'punc_count'] = pc
    
    text_l = text.lower()
    word_tokens = word_tokenize(text_l)
    stop_w = [word for word in word_tokens if word in stop_words]
    percent_stop = round((len(stop_w)/len(word_tokens)), 2)
    yelp.at[i, 'percent_stop'] = percent_stop
    
    filtered_sentence = []
    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(w) 
    Stem_words = []
    for w in filtered_sentence:
        rootWord=ps.stem(w)
        Stem_words.append(rootWord)
    yelp.at[i, 'length_preprocessed_review'] = len(Stem_words)

100%|██████████| 99579/99579 [04:25<00:00, 374.97it/s]


In [0]:
#sentence count 
num_sent = []
for i in yelp['text']:
    num_sent.append(len(sent_tokenize(i)))
yelp['sent_count'] = num_sent

#word count without puncuation
tokenizer_words = RegexpTokenizer(r'\w+')
num_words = []
for i in yelp['text']:
    num_words.append(len(tokenizer_words.tokenize(i)))
yelp['word_count'] = num_words


#count exclamations in reviews
tokenizer_exclamation = RegexpTokenizer(r'\!')
num_exclaim = []
for i in yelp['text']:
    num_exclaim.append(len(tokenizer_exclamation.tokenize(i)))
yelp['exclaim_count'] = num_exclaim

#create average sentence length variable
yelp['avg_sent']= yelp['word_count']/yelp['sent_count']


In [0]:
yelp = yelp.drop(['length_preprocessed_review','sent_count'],axis=1)


In [0]:
y = yelp['pos_class']
X_train, X_test, y_train, y_test = train_test_split(yelp, y, test_size=0.3, random_state=5, shuffle=True)

In [0]:
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
X_train = X_train.reset_index(drop=True)

In [0]:
X_train_scaled = X_train[['punc_count','percent_stop','word_count','exclaim_count','avg_sent']]
X_test_scaled = X_test[['punc_count','percent_stop','word_count','exclaim_count','avg_sent']]
X_train = X_train.drop(['punc_count','percent_stop','word_count','exclaim_count','avg_sent'],axis=1)
X_test = X_test.drop(['punc_count','percent_stop','word_count','exclaim_count','avg_sent'],axis=1)

In [0]:
columns = ['punc_count','percent_stop','word_count','exclaim_count','avg_sent']

In [0]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_scaled)
X_test_scaled = scaler.transform(X_test_scaled)

In [0]:
X_train_scaled = pd.DataFrame(X_train_scaled,columns=columns )
X_test_scaled = pd.DataFrame(X_test_scaled,columns=columns )

In [0]:
X_train = X_train.join(X_train_scaled)
X_test = X_test.join(X_test_scaled)

In [0]:
#methods to vectorize characters

#second does only character grams (second vectorizer would need to be set up for words)
countvec = TfidfVectorizer(analyzer='char', ngram_range=(2, 3),min_df = 0.05)
X_train_count_grams = countvec.fit_transform(X_train['clean_text'])


In [115]:
X_train_count_grams

<69705x2086 sparse matrix of type '<class 'numpy.float64'>'
	with 34179938 stored elements in Compressed Sparse Row format>

In [0]:
gram_labels = countvec.get_feature_names()
gram_matrix = X_train_count_grams.todense()

In [0]:
gram_matrix = pd.DataFrame(gram_matrix, columns=gram_labels)
gram_matrix = gram_matrix.add_suffix('_grams')
X_train = X_train.join(gram_matrix)

In [0]:
#test data

X_test_count_grams = countvec.transform(X_test['clean_text'])

In [119]:
X_test_count_grams

<29874x2086 sparse matrix of type '<class 'numpy.float64'>'
	with 14486488 stored elements in Compressed Sparse Row format>

In [0]:
gram_labels = countvec.get_feature_names()
gram_matrix = X_test_count_grams.todense()

In [0]:
gram_matrix = pd.DataFrame(gram_matrix, columns=gram_labels)
gram_matrix = gram_matrix.add_suffix('_grams')
X_test = X_test.join(gram_matrix)

In [0]:
cv_pos = TfidfVectorizer(vocabulary=pos_words,min_df = 0.001)
train_pos = cv_pos.fit_transform(X_train['text_one_hot'])

test_pos = cv_pos.transform(X_test['text_one_hot'])                           

cv_neg = TfidfVectorizer(vocabulary=neg_words,min_df = 0.001)
train_neg = cv_neg.fit_transform(X_train['text_one_hot'])

test_neg = cv_neg.transform(X_test['text_one_hot'])

In [0]:
train_pos = train_pos.todense()
test_pos = test_pos.todense()

train_neg = train_neg.todense()
test_neg = test_neg.todense()

train_pos = pd.DataFrame(train_pos,columns=cv_pos.get_feature_names())
train_pos = train_pos.add_suffix('_pos')
train_neg = pd.DataFrame(train_neg,columns=cv_neg.get_feature_names())
train_neg = train_neg.add_suffix('_neg')

test_pos = pd.DataFrame(test_pos,columns=cv_pos.get_feature_names())
test_pos = test_pos.add_suffix('_pos')
test_neg = pd.DataFrame(test_neg,columns=cv_neg.get_feature_names())
test_neg = test_neg.add_suffix('_neg')

In [0]:
X_train = X_train.join(train_pos)
X_train = X_train.join(train_neg)

X_test = X_test.join(test_pos)
X_test = X_test.join(test_neg)



## ML Models

In [0]:

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.compose import ColumnTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import classification_report, accuracy_score, f1_score

In [0]:


X_train = X_train.drop(['text','pos_class','clean_text','lemmas','text_one_hot'], axis=1)
X_test = X_test.drop(['text','pos_class','clean_text','lemmas','text_one_hot'], axis=1)
columns = X_train.columns

In [0]:
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()

In [0]:
X_train = csr_matrix(X_train)
X_test = csr_matrix(X_test)

In [0]:
# Extract information gain of each variable

res = dict(zip(columns,
               mutual_info_classif(X_train, y_train, discrete_features=True,random_state=5)
               ))


In [0]:
import json
with open('yelp_minfo.txt', 'w') as file:
     file.write(json.dumps(res))

files.download('yelp_minfo.txt')

In [131]:
sgd = SGDClassifier(verbose=1, random_state=5,n_jobs=-1)
sgd.fit(X_train,y_train)

-- Epoch 1
Norm: 34.65, NNZs: 4574, Bias: -0.654822, T: 69705, Avg. loss: 0.586050
Total training time: 0.14 seconds.
-- Epoch 2
Norm: 30.38, NNZs: 4736, Bias: -0.673144, T: 139410, Avg. loss: 0.211319
Total training time: 0.29 seconds.
-- Epoch 3
Norm: 28.83, NNZs: 4790, Bias: -0.670190, T: 209115, Avg. loss: 0.191744
Total training time: 0.43 seconds.
-- Epoch 4
Norm: 28.09, NNZs: 4817, Bias: -0.668891, T: 278820, Avg. loss: 0.184093
Total training time: 0.58 seconds.
-- Epoch 5
Norm: 27.71, NNZs: 4828, Bias: -0.663007, T: 348525, Avg. loss: 0.179006
Total training time: 0.72 seconds.
-- Epoch 6
Norm: 27.44, NNZs: 4840, Bias: -0.655483, T: 418230, Avg. loss: 0.176024
Total training time: 0.87 seconds.
-- Epoch 7
Norm: 27.29, NNZs: 4844, Bias: -0.651511, T: 487935, Avg. loss: 0.174119
Total training time: 1.02 seconds.
-- Epoch 8
Norm: 27.15, NNZs: 4847, Bias: -0.647863, T: 557640, Avg. loss: 0.172232
Total training time: 1.16 seconds.
-- Epoch 9
Norm: 27.10, NNZs: 4850, Bias: -0.6418

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=-1, penalty='l2',
              power_t=0.5, random_state=5, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=1, warm_start=False)

In [132]:
y_pred = sgd.predict(X_test)
print('accuracy %s' % accuracy_score(y_test,y_pred))

accuracy 0.932449621744661


In [0]:
# uploaded = files.upload()

In [0]:
# imdb = pd.read_csv(io.StringIO(uploaded['imdb_dataset.csv'].decode('utf-8')))

In [0]:
imdb = pd.read_csv('imdb_dataset.csv')

In [0]:
one_hot = pd.get_dummies(imdb['sentiment'])
imdb = imdb.join(one_hot)
imdb = imdb.drop(['sentiment','negative'], axis=1)

imdb = imdb.rename(columns={'review':'text','positive':'pos_class'})


In [137]:
clean_text = []
for t in tqdm(imdb['text']):
    temp = []
    t = t.replace('\n','')
    t = t.replace('<br','')
    t = t.replace('/>','')
    temp.append(t)
    clean_text.append(temp)
clean_text = pd.DataFrame(clean_text, columns=['clean_text'])
imdb = imdb.join(clean_text)

100%|██████████| 50000/50000 [00:00<00:00, 164236.73it/s]


In [138]:
#create lemmas feature
lemmas = [lemmatize_text(imdb.text[x]) for x in tqdm(range(len(imdb['text']))) ]

100%|██████████| 50000/50000 [02:21<00:00, 352.16it/s]


In [0]:
imdb['lemmas'] = lemmas

In [140]:
#create binary feature of if negations are present
negations = []
for word in tqdm(imdb['lemmas']):
    count =0
    for w in word:
        if w.lower() in negate:
            count+=1
    if count >= 1:
        negations.append(1)
    else:
        negations.append(0)
            

100%|██████████| 50000/50000 [00:28<00:00, 1768.18it/s]


In [0]:
imdb['negate'] = negations

In [0]:
one_hot_text = []
for t in imdb['lemmas']:
    string = ""
    for i in t:
        temp = []
        string = string + " " + i
    temp.append(string)
    one_hot_text.append(temp)

In [0]:
one_hot_text_imdb = pd.DataFrame(one_hot_text, columns=['text_one_hot'])
imdb = imdb.join(one_hot_text_imdb)

In [144]:
ps = PorterStemmer()
for i,r in tqdm(imdb.iterrows(), total=imdb.shape[0]):
    text = r['text']
    counts = Counter(text)
    punctuation_counts = {k:v for k, v in counts.items() if k in punctuation}
    pc = sum(punctuation_counts.values())
    imdb.at[i, 'punc_count'] = pc
    
    text_l = text.lower()
    word_tokens = word_tokenize(text_l)
    stop_w = [word for word in word_tokens if word in stop_words]
    percent_stop = round((len(stop_w)/len(word_tokens)), 2)
    imdb.at[i, 'percent_stop'] = percent_stop
    
    filtered_sentence = []
    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(w) 
    Stem_words = []
    for w in filtered_sentence:
        rootWord=ps.stem(w)
        Stem_words.append(rootWord)
    imdb.at[i, 'length_preprocessed_review'] = len(Stem_words)

100%|██████████| 50000/50000 [03:54<00:00, 212.81it/s]


In [0]:
#sentence count 
num_sent = []
for i in imdb['text']:
    num_sent.append(len(sent_tokenize(i)))
imdb['sent_count'] = num_sent

#word count without puncuation
tokenizer_words = RegexpTokenizer(r'\w+')
num_words = []
for i in imdb['text']:
    num_words.append(len(tokenizer_words.tokenize(i)))
imdb['word_count'] = num_words

#count exclamations in reviews
tokenizer_exclamation = RegexpTokenizer(r'\!')
num_exclaim = []
for i in imdb['text']:
    num_exclaim.append(len(tokenizer_exclamation.tokenize(i)))
imdb['exclaim_count'] = num_exclaim

#create average sentence length variable
imdb['avg_sent']= imdb['word_count']/imdb['sent_count']


In [0]:
imdb = imdb.drop(['length_preprocessed_review','sent_count'],axis=1)


In [0]:
y = imdb['pos_class']
X_train_imdb, X_test_imdb, y_train_imdb, y_test_imdb = train_test_split(imdb, y, test_size=0.3, random_state=5, shuffle=True)

In [0]:
y_train_imdb = y_train_imdb.reset_index(drop=True)
y_test_imdb = y_test_imdb.reset_index(drop=True)
X_test_imdb = X_test_imdb.reset_index(drop=True)
X_train_imdb = X_train_imdb.reset_index(drop=True)

In [149]:
X_train_imdb.head()

,text,pos_class,clean_text,lemmas,negate,text_one_hot,punc_count,percent_stop,word_count,exclaim_count,avg_sent
0,"As a teenager, I was pretty into the whole Big...",0,"As a teenager, I was pretty into the whole Big...","[as, a, teenager, i, be, pretty, into, the, wh...",1,as a teenager i be pretty into the whole bigf...,69.0,0.42,427,1,32.846154
1,Lulu (Louise Brooks) works as a typist and is ...,0,Lulu (Louise Brooks) works as a typist and is ...,"[lulu, louise, brook, work, as, a, typist, and...",1,lulu louise brook work as a typist and be mis...,34.0,0.44,200,0,16.666667
2,The whole Biker Movie genre has to be made up ...,0,The whole Biker Movie genre has to be made up ...,"[the, whole, biker, movie, genre, have, to, be...",1,the whole biker movie genre have to be make u...,16.0,0.43,114,1,16.285714
3,His John Dark-Kevin Connor fantasy adventures ...,1,His John Dark-Kevin Connor fantasy adventures ...,"[his, john, connor, fantasy, adventure, be, a,...",1,his john connor fantasy adventure be a mainst...,48.0,0.34,240,1,40.000000
4,Not since Caligula have I considered turning o...,0,Not since Caligula have I considered turning o...,"[not, since, caligula, have, i, consider, turn...",1,not since caligula have i consider turn off t...,48.0,0.38,141,0,28.200000


In [0]:
X_train_scaled_imdb = X_train_imdb[['punc_count','percent_stop','word_count','exclaim_count','avg_sent']]
X_test_scaled_imdb = X_test_imdb[['punc_count','percent_stop','word_count','exclaim_count','avg_sent']]
X_train_imdb = X_train_imdb.drop(['punc_count','percent_stop','word_count','exclaim_count','avg_sent'],axis=1)
X_test_imdb = X_test_imdb.drop(['punc_count','percent_stop','word_count','exclaim_count','avg_sent'],axis=1)

In [0]:
columns = ['punc_count','percent_stop','word_count','exclaim_count','avg_sent']

In [0]:
scaler = StandardScaler()
X_train_scaled_imdb = scaler.fit_transform(X_train_scaled_imdb)
X_test_scaled_imdb = scaler.transform(X_test_scaled_imdb)

In [0]:
X_train_scaled_imdb = pd.DataFrame(X_train_scaled_imdb,columns=columns )
X_test_scaled_imdb = pd.DataFrame(X_test_scaled_imdb,columns=columns )

In [0]:
X_train_imdb = X_train_imdb.join(X_train_scaled_imdb)
X_test_imdb = X_test_imdb.join(X_test_scaled_imdb)

In [0]:
countvec_imdb = TfidfVectorizer(analyzer='char', ngram_range=(2, 3),min_df = 0.05)
X_train_count_grams_imdb = countvec_imdb.fit_transform(X_train_imdb['clean_text'])


In [156]:
X_train_count_grams_imdb


<35000x3012 sparse matrix of type '<class 'numpy.float64'>'
	with 28907644 stored elements in Compressed Sparse Row format>

In [0]:
gram_labels_imdb = countvec_imdb.get_feature_names()
gram_matrix_imdb = X_train_count_grams_imdb.todense()

In [0]:
gram_matrix_imdb = pd.DataFrame(gram_matrix_imdb, columns=gram_labels_imdb)
gram_matrix_imdb = gram_matrix_imdb.add_suffix('_grams')
X_train_imdb = X_train_imdb.join(gram_matrix_imdb)

In [0]:
#test data

X_test_count_grams_imdb = countvec_imdb.transform(X_test_imdb['clean_text'])

In [160]:
X_test_count_grams_imdb

<15000x3012 sparse matrix of type '<class 'numpy.float64'>'
	with 12359320 stored elements in Compressed Sparse Row format>

In [0]:
gram_labels_imdb = countvec_imdb.get_feature_names()
gram_matrix_imdb = X_test_count_grams_imdb.todense()

In [0]:
gram_matrix_imdb = pd.DataFrame(gram_matrix_imdb, columns=gram_labels_imdb)
gram_matrix_imdb = gram_matrix_imdb.add_suffix('_grams')
X_test_imdb = X_test_imdb.join(gram_matrix_imdb)

In [0]:
cv_pos_imdb = TfidfVectorizer(vocabulary=pos_words,min_df = 0.001)
train_pos_imdb = cv_pos_imdb.fit_transform(X_train_imdb['text_one_hot'])

test_pos_imdb = cv_pos_imdb.transform(X_test_imdb['text_one_hot'])                           

cv_neg_imdb = TfidfVectorizer(vocabulary=neg_words,min_df = 0.001)
train_neg_imdb = cv_neg_imdb.fit_transform(X_train_imdb['text_one_hot'])

test_neg_imdb = cv_neg_imdb.transform(X_test_imdb['text_one_hot'])

In [0]:
train_pos_imdb = train_pos_imdb.todense()
test_pos_imdb = test_pos_imdb.todense()

train_neg_imdb = train_neg_imdb.todense()
test_neg_imdb = test_neg_imdb.todense()

train_pos_imdb = pd.DataFrame(train_pos_imdb,columns=cv_pos_imdb.get_feature_names())
train_pos_imdb = train_pos_imdb.add_suffix('_pos')
train_neg_imdb = pd.DataFrame(train_neg_imdb,columns=cv_neg_imdb.get_feature_names())
train_neg_imdb = train_neg_imdb.add_suffix('_neg')

test_pos_imdb = pd.DataFrame(test_pos_imdb,columns=cv_pos_imdb.get_feature_names())
test_pos_imdb = test_pos_imdb.add_suffix('_pos')
test_neg_imdb = pd.DataFrame(test_neg_imdb,columns=cv_neg_imdb.get_feature_names())
test_neg_imdb = test_neg_imdb.add_suffix('_neg')

In [0]:
X_train_imdb = X_train_imdb.join(train_pos_imdb)
X_train_imdb = X_train_imdb.join(train_neg_imdb)

X_test_imdb = X_test_imdb.join(test_pos_imdb)
X_test_imdb = X_test_imdb.join(test_neg_imdb)



In [0]:
X_train_imdb = X_train_imdb.drop(['text','pos_class','clean_text','lemmas','text_one_hot'], axis=1)
X_test_imdb = X_test_imdb.drop(['text','pos_class','clean_text','lemmas','text_one_hot'], axis=1)
columns = X_train_imdb.columns


In [0]:
X_train_imdb = X_train_imdb.to_numpy()
X_test_imdb = X_test_imdb.to_numpy()

In [0]:
X_train_imdb = csr_matrix(X_train_imdb)
X_test_imdb = csr_matrix(X_test_imdb)

In [0]:
# Extract information gain of each variable
res = dict(zip(columns,
               mutual_info_classif(X_train_imdb, y_train_imdb, discrete_features=True,random_state=5)
               ))


In [0]:
import json
with open('imdb_minfo.txt', 'w') as file:
     file.write(json.dumps(res))

files.download('imdb_minfo.txt')

In [171]:
sgd = SGDClassifier(verbose=1, random_state=5,n_jobs=-1)
sgd.fit(X_train_imdb,y_train_imdb)

-- Epoch 1
Norm: 47.93, NNZs: 6265, Bias: 0.399570, T: 35000, Avg. loss: 1.409895
Total training time: 0.09 seconds.
-- Epoch 2
Norm: 39.22, NNZs: 6359, Bias: 0.443803, T: 70000, Avg. loss: 0.473163
Total training time: 0.18 seconds.
-- Epoch 3
Norm: 36.28, NNZs: 6381, Bias: 0.474391, T: 105000, Avg. loss: 0.381718
Total training time: 0.27 seconds.
-- Epoch 4
Norm: 34.69, NNZs: 6395, Bias: 0.486245, T: 140000, Avg. loss: 0.350076
Total training time: 0.36 seconds.
-- Epoch 5
Norm: 33.92, NNZs: 6400, Bias: 0.503256, T: 175000, Avg. loss: 0.329823
Total training time: 0.46 seconds.
-- Epoch 6
Norm: 33.44, NNZs: 6405, Bias: 0.516207, T: 210000, Avg. loss: 0.312922
Total training time: 0.56 seconds.
-- Epoch 7
Norm: 32.90, NNZs: 6406, Bias: 0.523632, T: 245000, Avg. loss: 0.304786
Total training time: 0.65 seconds.
-- Epoch 8
Norm: 32.65, NNZs: 6409, Bias: 0.528856, T: 280000, Avg. loss: 0.299994
Total training time: 0.74 seconds.
-- Epoch 9
Norm: 32.46, NNZs: 6410, Bias: 0.534272, T: 315

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=-1, penalty='l2',
              power_t=0.5, random_state=5, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=1, warm_start=False)

In [172]:
y_pred_imdb = sgd.predict(X_test_imdb)
print('accuracy %s' % accuracy_score(y_test_imdb,y_pred_imdb))

accuracy 0.8747333333333334
